In [17]:
import numpy as np
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from importlib import reload
import visualise
reload(visualise)
from visualise import get_tsne_points, do_inference
from constants import *

init_notebook_mode(connected=True)

In [18]:
def plot_points(points: np.ndarray):
    xs = points[:, 0]
    ys = points[:, 1]
    fig = px.scatter(x=xs, y=ys)
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        template='seaborn'
        
    )
    fig.show()

In [19]:
import warnings
warnings.filterwarnings('ignore')


points2d, labels = do_inference(f"{DATA_DIR}/Labels/all_pruned.csv", "saved_models/mbt_student_val_loss_3395.08605.pth")

Loading audio layers...
torch.Size([1, 1, 1024]) torch.Size([1, 184, 1024])
Loaded successfully in 3.8s
Loading video layers...
Loaded successfully in 3.9s
Reset trainable parameters of layer = 8.norm1
Reset trainable parameters of layer = 8.attn.qkv
Reset trainable parameters of layer = 8.attn.proj
Reset trainable parameters of layer = 8.norm2
Reset trainable parameters of layer = 8.mlp.fc1
Reset trainable parameters of layer = 8.mlp.fc2
Reset trainable parameters of layer = 9.norm1
Reset trainable parameters of layer = 9.attn.qkv
Reset trainable parameters of layer = 9.attn.proj
Reset trainable parameters of layer = 9.norm2
Reset trainable parameters of layer = 9.mlp.fc1
Reset trainable parameters of layer = 9.mlp.fc2
Reset trainable parameters of layer = 8.norm1
Reset trainable parameters of layer = 8.attn.qkv
Reset trainable parameters of layer = 8.attn.proj
Reset trainable parameters of layer = 8.norm2
Reset trainable parameters of layer = 8.mlp.fc1
Reset trainable parameters of l

  0%|          | 0/14 [00:00<?, ?it/s]/root/depmbt/data.py:32: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning:

PySoundFile failed. Trying audioread instead.

/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning:

PySoundFile failed. Trying audioread instead.

/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning:

PySoundFile failed. Trying audioread instead.

/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning:

PySoundFile failed. Trying audioread instead.

/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning:

PySoundFil

RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/root/miniconda/envs/depmbt/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/root/miniconda/envs/depmbt/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/root/depmbt/vitmbt.py", line 193, in forward
    a  = self.multimodal_audio[i](a)
  File "/root/miniconda/envs/depmbt/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/root/depmbt/vision_transformer.py", line 153, in forward
    x = x + self.drop_path1(self.ls1(self.attn(self.norm1(x))))
  File "/root/miniconda/envs/depmbt/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/root/depmbt/vision_transfo

In [13]:
test = np.random.rand(300, 8, 4, 1024)
points = get_tsne_points(test)
plot_points(points)